# Multimodal Model – Combining Time-Series, Static Variables, Images, and Text

In this lab, we will explore multimodal models that integrate time-series data, static variables, images, and text, leveraging all the datasets we have used previously.

First, we need to install and import the necessary libraries.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

bookPath = '/content/drive/MyDrive/Colab Notebooks/DL in Banking Book/DeepLearningInBankingBook/TextBook_Lab'

In [ ]:
bookPath = '/content/drive/MyDrive/Colab Notebooks/DL in Banking Book/DeepLearningInBankingBook/TextBook_Lab'

In [ ]:
# Install necessasary packages, if not done before
!pip install transformers evaluate accelerate hf_xet

In [ ]:
# Imports
import numpy as np
import os
import pandas as pd
import random
# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, auc

# Plots
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
%matplotlib inline

# Import Pytorch lybraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.optim.lr_scheduler import _LRScheduler

# Huggingface
import transformers
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import pipeline
from transformers import set_seed
from datasets import load_dataset, Dataset, Value, ClassLabel, Features, load_from_disk
import evaluate

## Cleaning Time Series data

Let's start by downloading the time-series dataset from Freddie Mac. This dataset provides monthly loan status updates covering the period from November 2021 to June 2024.

In [ ]:
!gdown --fuzzy 'https://drive.google.com/file/d/1cte8MKaSdt3LrqXSUo9oGTe2hnkz8JsU/view?usp=sharing'

In [ ]:
# Read the data
df = pd.read_csv('Multimodal_Lab_sampled.csv', low_memory=False)
df

We assign a value of 1 to loans that were delinquent at least once in the last three months (April to June 2024). Our goal is to predict whether a loan will become delinquent in the next quarter.

In [ ]:
# Convert REPORTING_PERIOD to datetime for sorting
df["REPORTING_PERIOD"] = pd.to_datetime(df["REPORTING_PERIOD"], format='%Y%m')

df["CURRENT_LOAN_DELINQUENCY_STATUS"] = pd.to_numeric(df["CURRENT_LOAN_DELINQUENCY_STATUS"], errors='coerce').fillna(1)

# Sort data by Loan Number and Reporting Period (chronological order)
df = df.sort_values(by=["LOAN_NUMBER", "REPORTING_PERIOD"])


# Identify loans that were delinquent at least once in the period 2024-04 to 2024-06
df["CURRENT_LOAN_DELINQUENCY_STATUS"] = (df["CURRENT_LOAN_DELINQUENCY_STATUS"] > 0).astype(int)
mask = (df["REPORTING_PERIOD"] >= "2024-04-01") & (df["REPORTING_PERIOD"] <= "2024-06-30")
df["target"] = df.groupby("LOAN_NUMBER")["CURRENT_LOAN_DELINQUENCY_STATUS"].transform(lambda x: x[mask].max()).astype(int)


# Remove the last three months (April 2024 - June 2024)
df = df[~df["REPORTING_PERIOD"].between(pd.Timestamp("2024-03-31"), pd.Timestamp("2024-06-02"))]
df

We normalize the data and One Hot Encode the labels using sklearn's OneHotEncode function. We will also set the class weights as we have done before.

In [ ]:
num_cols = [
    "CURRENT_ACTUAL_UPB",
    "LOAN_AGE",
    "REMAINING_MONTHS",
    "CURRENT_INTEREST_RATE",
    "CURRENT_NON_INTEREST_BEARING_UPB",
    "ESTIMATED_LOAN_TO_VALUE",
    "INTEREST_BEARING_UPB",
    "CURRENT_LOAN_DELINQUENCY_STATUS"
]

# Group by loan number for sequence creation
grouped = df.groupby("LOAN_NUMBER")

# Store sequences and targets
X_dict = {}
y_dict = {}

for loan_id, group in grouped:
    group = group.sort_values("REPORTING_PERIOD")  # Sort each loan's time series

    # Extract features as separate sequences
    X_dict[loan_id] = {col: group[col].values for col in num_cols}

    # Assign existing target value (no need to recalculate)
    y_dict[loan_id] = group["target"].iloc[0]



# Perform stratified split
# Convert dictionary to DataFrame
np.random.seed(42)
x_df = pd.DataFrame.from_dict(X_dict, orient="index")

# Map target values from y_dict
x_df["target"] = x_df.index.map(y_dict)

# Perform stratified split
train_idx, test_idx = train_test_split(
    x_df.index, test_size=0.2, stratify=x_df["target"], random_state=42
)

# Assign 'if_test' column based on stratified split
x_df["if_test"] = 0  # Default to training
x_df.loc[test_idx, "if_test"] = 1  # Mark test samples

In [ ]:
# Function to scale each feature separately
def scale_dataframe_sequences(x_df, test_var=None):
    scaler = StandardScaler()

    # Split data into train and test
    if test_var is not None:
        x_train = x_df.loc[test_var == 0, :]
        x_test = x_df.loc[test_var == 1, :]
    else:
        x_train = x_df

    scaled_data = {}
    scaled_data_test = {}

    # Get sequence length from first row
    seq_len = len(next(iter(x_train.iloc[0])))

    for column in num_cols:
        # Stack time series data for each loan into a 2D array
        data = np.stack(x_train[column].values).reshape(-1, seq_len)
        if column != "CURRENT_LOAN_DELINQUENCY_STATUS":
          scaled_data[column] = scaler.fit_transform(data).reshape(-1, 1, seq_len).tolist()
        else:
          scaled_data[column] = data.reshape(-1, 1, seq_len).tolist()

        # Scale test set if provided
        if test_var is not None:
            data_test = np.stack(x_test[column].values).reshape(-1, seq_len)
            if column != "CURRENT_LOAN_DELINQUENCY_STATUS":
              scaled_data_test[column] = scaler.transform(data_test).reshape(-1, 1, seq_len).tolist()
            else:
              scaled_data_test[column] = data_test.reshape(-1, 1, seq_len).tolist()

    # Create new DataFrames with scaled data
    scaled_df = pd.DataFrame(scaled_data)
    scaled_df = scaled_df.map(lambda x: np.array(x[0]))

    if test_var is not None:
        scaled_df_test = pd.DataFrame(scaled_data_test)
        scaled_df_test = scaled_df_test.map(lambda x: np.array(x[0]))
        return scaled_df, scaled_df_test
    else:
        return scaled_df, _

# Normalize train and test sets
x_train_val, x_test = scale_dataframe_sequences(x_df[num_cols], x_df["if_test"])

# One-hot encode the target variable
enc = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
y_train_val = enc.fit_transform(x_df.loc[x_df["if_test"] == 0, "target"].values.reshape(-1, 1))[:, 1]
y_test = enc.transform(x_df.loc[x_df["if_test"] == 1, "target"].values.reshape(-1, 1))[:, 1]

# Compute class weights
pos_weight = np.sum(1 - y_train_val) / np.sum(y_train_val)
pos_weight = torch.tensor(pos_weight, dtype=torch.float32)

In [ ]:
x_train_val

## Cleaning static variables

Next, we will clean static features such as credit score, property type, and occupancy status, which were recorded at the time of the loan application.

In [ ]:
!gdown --fuzzy 'https://drive.google.com/file/d/1lF1n0Ue-emaflJabOfmQI6CT0D3qAIYK/view?usp=sharing'

In [ ]:
# Load filtered static dataset
static_df = pd.read_csv("Multimodal_Lab_Origin_sampled.csv", dtype=str)
static_df

We normalize numeric variables and one-hot encode categorical variables to ensure consistency in the dataset.

In [ ]:
# Identify numeric and categorical columns
numeric_cols = ["CREDIT_SCORE", "CLTV", "DTI_RATIO", "ORIGINAL_UPB", "ORIGINAL_LOAN_TERM", "MI_PERCENTAGE"]  # Excluding "NUMBER_OF_BORROWERS"
categorical_cols = ["FIRST_TIME_HOMEBUYER", "OCCUPANCY_STATUS", "PROPERTY_TYPE"]


static_df["ZIP3"] = static_df["ZIP3"].astype(str).str[:-2]
static_df[numeric_cols] = static_df[numeric_cols].astype(float)
static_df["NUMBER_OF_BORROWERS"] = static_df["NUMBER_OF_BORROWERS"].astype(int)

In [ ]:
# Add LOAN_NUMBER back before merging
x_df["LOAN_NUMBER"] = x_df.index  # Assign index (loan numbers) back to x_df

# Assign LOAN_NUMBER to train and test sets
loan_numbers_train = x_df.loc[x_df["if_test"] == 0, "LOAN_NUMBER"]
loan_numbers_test = x_df.loc[x_df["if_test"] == 1, "LOAN_NUMBER"]

# Split static data into train and test based on LOAN_NUMBER
static_train_val = static_df[static_df["LOAN_NUMBER"].isin(loan_numbers_train)]
static_test = static_df[static_df["LOAN_NUMBER"].isin(loan_numbers_test)]

# --- SCALE NUMERIC FEATURES ---
scaler = StandardScaler()
static_train_val[numeric_cols] = scaler.fit_transform(static_train_val[numeric_cols])  # Fit on train
static_test[numeric_cols] = scaler.transform(static_test[numeric_cols])  # Apply to test

# --- ONE-HOT ENCODE CATEGORICAL FEATURES ---
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded_train = encoder.fit_transform(static_train_val[categorical_cols])  # Fit on train
encoded_test = encoder.transform(static_test[categorical_cols])  # Apply to test

# Convert to DataFrame
encoded_train_df = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out(categorical_cols), index=static_train_val.index)
encoded_test_df = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out(categorical_cols), index=static_test.index)

We then merge the cleaned static variables with the time-series table to create a unified dataset for modeling.

In [ ]:
# Preserve LOAN_NUMBER and NUMBER_OF_BORROWERS (without scaling)
static_train_final = pd.concat([
    static_train_val[["LOAN_NUMBER", "NUMBER_OF_BORROWERS","MSA","ZIP3"]],  # Keep unscaled
    static_train_val[numeric_cols],  # Already scaled numeric features
    encoded_train_df  # One-hot encoded categorical features
], axis=1)

static_test_final = pd.concat([
    static_test[["LOAN_NUMBER", "NUMBER_OF_BORROWERS","MSA","ZIP3"]],  # Keep unscaled
    static_test[numeric_cols],  # Already scaled numeric features
    encoded_test_df  # One-hot encoded categorical features
], axis=1)

print(f"Processed Train Static Data Shape: {static_train_final.shape}")
print(f"Processed Test Static Data Shape: {static_test_final.shape}")

In [ ]:
# Convert index to DataFrame to allow merging
x_train_val = x_train_val.copy()
x_test = x_test.copy()

# Add LOAN_NUMBER column before merging
x_train_val["LOAN_NUMBER"] = loan_numbers_train.values
x_test["LOAN_NUMBER"] = loan_numbers_test.values

# Merge static data with LSTM sequences
x_train_val = x_train_val.merge(static_train_final, on="LOAN_NUMBER", how="left")
x_test = x_test.merge(static_test_final, on="LOAN_NUMBER", how="left")

# Drop LOAN_NUMBER after merging
x_train_val.drop(columns=["LOAN_NUMBER"], inplace=True)
x_test.drop(columns=["LOAN_NUMBER"], inplace=True)

print(f"x_train Shape After Merge: {x_train_val.shape}")
print(f"x_test Shape After Merge: {x_test.shape}")

## Merging image dataset

Next, we will merge the image mapping dataset using MSA (Metropolitan Statistical Area) and ZIP3 (the first three digits of the ZIP code) to align LiDAR images with the corresponding loan records.

In [ ]:
!gdown --fuzzy 'https://drive.google.com/file/d/1cHLNcnTLdaKtedM_Uz-xmYaXXml0Ws-D/view?usp=sharing'

In [ ]:
!unzip usgs_lidar.zip -d usgs_lidar

In [ ]:
!gdown --fuzzy 'https://drive.google.com/file/d/1QyzNtHVurvrXvHG1j2oeq7rwfC6vnA1h/view?usp=sharing'

In [ ]:
image_df = pd.read_csv('msa_zip3_mapping_cleaned.csv', dtype=str)
image_df

In [ ]:
# Merge static data with LSTM sequences
x_train_val = x_train_val.merge(image_df, on=["MSA","ZIP3"], how="left")
x_test = x_test.merge(image_df, on=["MSA","ZIP3"], how="left")

# Drop LOAN_NUMBER after merging
x_train_val.drop(columns=["MSA", "ZIP3"], inplace=True)
x_test.drop(columns=["MSA", "ZIP3"], inplace=True)

Finally, we have established a comprehensive dataset that integrates time-series data, static variables, and image mapping, providing a unified foundation for our multimodal analysis.

In [ ]:
x_train_val

##  Get fixed textual data

We will use the most recent Federal Reserve speech from March 2024 as the fixed textual data for our analysis.

**Scenario**:

You are a data scientist at a bank, analyzing loan risk. After watching the Federal Reserve speech from March 2024, you aim to leverage multiple datasets—including time-series data from the loan dataset, static features from loan applications, LiDAR images, and the FED speech—to predict whether a loan will become delinquent in the next quarter.

In [ ]:
!gdown --fuzzy 'https://drive.google.com/file/d/1uVt9BC2tgr-MWrFZvYvA_I8IzTabNZtL/view?usp=sharing'

In [ ]:
fed_speech = pd.read_csv("/content/fed_speeches.csv", delimiter=",", on_bad_lines="skip", engine="python")

In [ ]:
# Create year and month columns
fed_speech['date'] = pd.to_datetime(fed_speech['date'], errors='coerce')
fed_speech["year"] = fed_speech["date"].dt.year  # Extract year
fed_speech["month"] = fed_speech["date"].dt.month  # Extract month
fed_speech

In [ ]:
fed_speech[(fed_speech.year == 2024) & (fed_speech.month == 3)]

In [ ]:
## taking the most recent speech
fixed_fed_speech = fed_speech[(fed_speech.year == 2024) & (fed_speech.month == 3)]["text"].values[-1]

In this step, we preprocess the speech text by **tokenizing, removing stopwords, and eliminating punctuation** to prepare the data for further analysis.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = set(stopwords.words("english"))


def clean_text(text):
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenize text
    table = str.maketrans('', '', string.punctuation)  # Create a table for removing punctuation
    filtered_tokens = [
        token.translate(table) for token in tokens
        if token.isalnum() and token not in stop_words  # Remove stop words here!
    ]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

fixed_fed_speech = clean_text(fixed_fed_speech)
fixed_fed_speech

## Preparing Dataloader for multimodal model

We create a **`Multimodal Dataset Class`**, integrating time-series data, static features, LiDAR images, and textual data.

In [ ]:
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from PIL import Image
import cv2

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
fixed_tokens = tokenizer(fixed_fed_speech, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
fixed_tokens = {key: val.squeeze(0) for key, val in fixed_tokens.items()}  # Store as dict

class MultimodalDataset(Dataset):
    def __init__(self, dataframe, y_values):
        """
        dataframe  : DataFrame containing time series, static variables, and LiDAR file paths
        fixed_text : Single fixed text string (e.g., FED speech that applies to all)
        tokenizer  : Transformer tokenizer for text processing
        max_length : Max token length for text processing
        """
        self.dataframe = dataframe.reset_index(drop=True)
        self.y_values = torch.tensor(y_values, dtype=torch.float32)  # Ensure target is float32


        # Define image transformation
        self.normalize_transform = transforms.Normalize(mean=[0.5], std=[0.5])


        # Identify column indices for time-series and static variables
        self.time_series_start = 0
        self.time_series_end = 8
        self.static_start = 8
        self.static_end = -1


        # Define image loading. We use CV2 for faster loading
    def load_image(self, image_path):
        """Load and preprocess image using OpenCV for speed."""
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        image = cv2.resize(image, (224, 224))  # Resize
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0) / 255.0  # Normalize 0-1
        return self.normalize_transform(image)



    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]

        #  Ensure time-series to tensor
        time_series = torch.tensor(
            np.array(row.iloc[self.time_series_start:self.time_series_end].tolist()),
            dtype=torch.float32
        )


        # Static variable to tensor
        static_features = torch.tensor(
            np.array(row.iloc[self.static_start:self.static_end].astype(float).values.tolist()),
            dtype=torch.float32
        )

        # Load LiDAR image (grayscale)
        image = self.load_image(row["LiDAR_File"])

        # Expand fixed text tokens to match batch size
        batch_text_tokens = fixed_tokens

        # Retrieve target
        target = self.y_values[idx]

        # Return dictionary (use precomputed text for all samples)
        return {
            "time_series": time_series,
            "static_features": static_features,
            "image_data": image,
            "text_data": batch_text_tokens,
            "target": target
        }

This code splits the dataset, creates PyTorch dataset instances, and prepares data loaders for training, validation, and testing in a multimodal deep learning model.

In [ ]:
def worker_init_fn(worker_id):
    """Ensure each worker has a different seed based on global seed."""
    seed = SEED + worker_id
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

In [ ]:
from sklearn.model_selection import train_test_split

# Train-val split
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.33, random_state=42, stratify=y_train_val)


SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # For multi-GPU
np.random.seed(SEED)
random.seed(SEED)

# Create dataset instances (pass y_train & y_test separately)
train_dataset = MultimodalDataset(x_train, y_train)
val_dataset = MultimodalDataset(x_val, y_val)
test_dataset = MultimodalDataset(x_test, y_test)

# Create PyTorch DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2,
                          worker_init_fn=worker_init_fn, pin_memory=True, persistent_workers=True, prefetch_factor=2)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

# Check batch structure
batch = next(iter(train_loader))
print(f"Time-series data shape: {batch['time_series'].shape}")  # (batch_size, num_features, seq_length)
print(f"Static feature shape: {batch['static_features'].shape}")  # (batch_size, num_static_features)
print(f"Text input IDs shape: {batch['text_data']['input_ids'].shape}")  # (batch_size, max_length)
print(f"Image data shape: {batch['image_data'].shape}")  # (batch_size, 1, 256, 256)
print(f"Target shape: {batch['target'].shape}")  # (batch_size,)

## Multimodal deep learning model - intermediate fusion

In this section, we will build a multimodal deep learning model that integrates four distinct data modalities:


*   Time Series Data: Loan performance history over time with Transformer Encoder.
*   Static Features: Borrower information, property details, and loan
*   LiDAR Images: LiDAR images processed using a CNN.
*   Textual Data: Federal Reserve speeches analyzed with DistilBERT.


To predict whether a loan will become delinquent in the following quarter, we will utilize a combination of Transformers for time-series data, CNNs for image processing, and a fusion layer to integrate information from all modalities effectively.

In [ ]:
import math
from transformers import DistilBertModel

### Positional Encoding Module
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + 0.1 * self.pe[:, :x.size(1), :]

### Time-Series Transformer Encoder with Positional Encoding
class TimeSeriesModel(nn.Module):
    def __init__(self, num_time_series_features, seq_length, dropout=0.1):
        super(TimeSeriesModel, self).__init__()

        self.proj = nn.Linear(num_time_series_features, 64)
        self.positional_encoding = PositionalEncoding(embed_dim=64, max_len=seq_length)

        encoder_layer = nn.TransformerEncoderLayer(d_model=64, nhead=4, dim_feedforward=128, dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)

        self.fc = nn.Sequential(
            nn.Linear(seq_length * 64, 128),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.proj(x)
        x = self.positional_encoding(x)
        x = self.transformer(x)
        x = x.reshape(x.shape[0], -1)
        return self.fc(x)

### CNN Model for LiDAR Images (BatchNorm + Dropout)
class LiDARCNN(nn.Module):
    def __init__(self, dropout=0.3):
        super(LiDARCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 56 * 56, 128),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

### Multimodal Model
class MultimodalDelinquencyModel(nn.Module):
    def __init__(self, num_time_series_features, seq_length, num_static_features, dropout=0.3):
        super(MultimodalDelinquencyModel, self).__init__()

        ### Time-Series Component (Transformer-Based with Positional Encoding)
        self.time_series_model = TimeSeriesModel(num_time_series_features, seq_length, dropout)

        ### Static Features Component (MLP)
        self.static_fc = nn.Sequential(
            nn.Linear(num_static_features, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        ### Text Component (DistilBERT)
        self.text_model = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.text_fc = nn.Sequential(
            nn.Linear(self.text_model.config.hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        ### LiDAR Image Component (CNN)
        self.image_model = LiDARCNN(dropout)

        ### Fusion Layer
        self.fusion_layer = nn.Sequential(
            nn.Linear(128 + 128 + 128 + 128, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 1)
        )

    def forward(self, time_series, static_features, text_inputs, image_data):
        ### Time-Series Processing
        time_series_out = self.time_series_model(time_series)

        ### Static Feature Processing
        static_out = self.static_fc(static_features)

        ### Text Processing (DistilBERT)
        text_out = self.text_model(**text_inputs).last_hidden_state[:, 0, :]
        text_out = self.text_fc(text_out)

        ### LiDAR Image Processing (CNN)
        image_out = self.image_model(image_data)

        ### Fusion
        fused = torch.cat([time_series_out, static_out, text_out, image_out], dim=1)
        output = self.fusion_layer(fused)

        return output  # Raw logits (for BCEWithLogitsLoss)


Now, let's begin training our multimodal deep learning model! Given its complexity, training may take some time.  

To improve efficiency, we are leveraging **mixed precision training** with `torch.amp.GradScaler`, which accelerates computations while reducing memory consumption.


In [ ]:
from sklearn.metrics import classification_report

SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # For multi-GPU
np.random.seed(SEED)
random.seed(SEED)


# Initialize model
num_time_series_features = 8
seq_length = len(x_train.iloc[0, 0])  # Adjust based on your time-series length
num_static_features = len(x_train.iloc[0, 8:-1])  # Adjust based on your dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultimodalDelinquencyModel(num_time_series_features, seq_length, num_static_features, dropout=0.3).to(device)

# Loss function & optimizer
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)  # Using BCEWithLogitsLoss
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

# Mixed Precision Training (AMP)
scaler = torch.amp.GradScaler('cuda')

# Best validation loss tracking
best_val_loss = float("inf")
checkpoint_path = "best_multimodal_model.pth"


### Training Function with Validation
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    global best_val_loss
    model.train()

    for epoch in range(num_epochs):
        total_train_loss = 0
        total_correct = 0
        total_samples = 0

        ### Training Loop
        for batch in train_loader:
            optimizer.zero_grad()

            # Move data to GPU
            time_series = batch["time_series"].to(device)
            static_features = batch["static_features"].to(device)
            image_data = batch["image_data"].to(device)
            text_inputs = {key: val.to(device) for key, val in batch["text_data"].items()}
            targets = batch["target"].to(device)

            with torch.amp.autocast('cuda'):  # Mixed Precision Training
                outputs = model(time_series, static_features, text_inputs, image_data).squeeze()
                loss = criterion(outputs, targets.float())

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_train_loss += loss.detach()  # Reduce unnecessary GPU sync

            # Convert logits to probabilities for accuracy calculation
            probs = torch.sigmoid(outputs)
            predictions = (probs > 0.5).float()
            total_correct += (predictions == targets).sum()
            total_samples += targets.size(0)

        train_acc = 100 * total_correct.cpu().item() / total_samples
        avg_train_loss = total_train_loss.item() / len(train_loader)

        ### Validation Loop
        model.eval()
        total_val_loss = 0
        correct, total = 0, 0

        with torch.no_grad():
            for batch in val_loader:
                time_series = batch["time_series"].to(device)
                static_features = batch["static_features"].to(device)
                image_data = batch["image_data"].to(device)
                text_inputs = {key: val.to(device) for key, val in batch["text_data"].items()}
                targets = batch["target"].to(device)

                outputs = model(time_series, static_features, text_inputs, image_data).squeeze()
                loss = criterion(outputs, targets.float())
                total_val_loss += loss.detach()

                # Convert logits to probabilities for accuracy calculation
                probs = torch.sigmoid(outputs)
                predictions = (probs > 0.5).float()
                correct += (predictions == targets).sum()
                total += targets.size(0)

        val_acc = 100 * correct.cpu().item() / total
        avg_val_loss = total_val_loss.item() / len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

        ### Save best model checkpoint
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model saved at epoch {epoch+1} with val loss: {avg_val_loss:.4f}")

    print("Training Complete!")

### One epoch takes 1.5 min with A100
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Training takes approximately **15 minutes**. However, we observe that further training may be beneficial, as the validation loss continues to decrease.


Now, let's proceed with evaluating the model.

In [ ]:
### Model Evaluation Function
def evaluate_model(model, test_loader, device):
    """
    Evaluates the model and plots the confusion matrix (normalized) and ROC curve.

    Args:
        model: Trained PyTorch model.
        test_loader: DataLoader with test samples.
        device: Device (cuda or cpu).

    Returns:
        all_labels, all_preds, all_probs
    """
    model.eval()
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            # Move data to device
            time_series = batch["time_series"].to(device)
            static_features = batch["static_features"].to(device)
            text_inputs = {key: val.to(device) for key, val in batch["text_data"].items()}
            image_data = batch["image_data"].to(device)
            targets = batch["target"].to(device)

            # Forward pass (raw logits)
            logits = model(time_series, static_features, text_inputs, image_data).squeeze()

            # Convert logits to probabilities using Sigmoid
            probs = torch.sigmoid(logits).cpu().numpy()

            # Store results
            all_probs.extend(probs)
            all_labels.extend(targets.cpu().numpy())

    # Convert lists to numpy arrays
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    # Compute binary predictions based on threshold 0.5
    all_preds = (all_probs >= 0.5).astype(int)

    return all_labels, all_preds, all_probs

In [ ]:
# Load the best model
model.load_state_dict(torch.load(checkpoint_path, weights_only=True))
print("Loaded best model for evaluation!")

# Evaluate on test data
all_labels, all_preds, all_probs = evaluate_model(model, test_loader, device)

Our model performs pretty well!

In [ ]:
### Confusion Matrix (Normalized)
cm = confusion_matrix(all_labels, all_preds)
cm_rates = cm.astype("float") / cm.sum(axis=1, keepdims=True)

plt.figure(figsize=(6, 5))
sns.heatmap(cm_rates, annot=True, fmt=".2f", cmap=sns.light_palette("seagreen", as_cmap=True),
                xticklabels=["No Delinquency", "Delinquent"],
                yticklabels=["No Delinquency", "Delinquent"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Normalized Confusion Matrix (Rates)")
plt.savefig(os.path.join(bookPath, "multimodal_confusion_matrix.pdf"))
plt.show()

In [ ]:
### ROC Curve & AUC Score
fpr, tpr, _ = roc_curve(all_labels, all_probs)
auc_score = roc_auc_score(all_labels, all_probs)

plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc_score:.3f})", linewidth=2)
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")  # Diagonal reference line
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve - Delinquency Prediction")
plt.legend(loc=4)
plt.grid()
plt.savefig(os.path.join(bookPath, "multimodal_roc_curves.pdf"))
plt.show()

### Classification Report
print("Classification Report:\n", classification_report(all_labels, all_preds, digits=4))

## Multimodal model with cross-attention

In this section, we implement a multimodal deep learning model that integrates information from multiple data sources using cross-attention mechanisms. Unlike traditional models that process different modalities independently, cross-attention allows the model to dynamically learn relationships between feature representations, improving predictive performance.

### Model Architecture

Our model processes four distinct modalities:

*  Time-Series Data: Loan performance over time, modeled using a Transformer with positional encoding.
*  Static Features: Borrower and loan characteristics, processed through a Multi-Layer Perceptron (MLP).
*  Text Data: Federal Reserve speeches, encoded using DistilBERT.
*  LiDAR Images: Spatial features extracted via a CNN.


To enhance feature interactions, cross-attention is applied at multiple levels:

*  Text ↔ Time-Series: Captures relationships between Federal Reserve statements and historical loan performance.
*  Static Features ↔ LiDAR Images: Links borrower characteristics with geographic-based insights.
*  Time-Series ↔ Static Features: Models how borrower attributes influence loan behavior over time.

### Fusion Strategy

After applying cross-attention, the model undergoes two levels of feature refinement:

*  Second-Level Cross-Attention: Further interactions between attended representations from different modalities.
*  Weighted Fusion Mechanism: A learnable attention-based fusion layer assigns adaptive importance to each attended feature representation before making a final prediction.
*  The fused representation is then passed through a fully connected prediction layer to estimate loan delinquency status.

In [ ]:
from transformers import DistilBertModel
import math

### Cross-Attention Module
class CrossAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=4, dropout=0.1):
        super(CrossAttention, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout)
        self.gate = nn.Linear(embed_dim * 2, 1)
        self.sigmoid = nn.Sigmoid()
        self.scale = nn.Parameter(torch.ones(1))  # Learnable scaling factor
        self.norm = nn.LayerNorm(embed_dim)  # Layer normalization

    def forward(self, x1, x2):
        """
        Cross-Attention between two modalities.
        Args:
            x1 (tensor): Query features (batch, feature_dim).
            x2 (tensor): Key and Value features (batch, feature_dim).
        Returns:
            tensor: Attended feature representation.
        """
        gate_value = self.sigmoid(self.gate(torch.cat([x1, x2], dim=-1)))  # Compute gate
        attn_output, _ = self.attention(x1.unsqueeze(0), x2.unsqueeze(0), x2.unsqueeze(0))
        return self.norm(self.scale * (gate_value * attn_output.squeeze(0) + (1 - gate_value) * x1))


### Positional Encoding Module
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()

        # Create a positional encoding matrix (max_len, embed_dim)
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # Compute sine and cosine functions for even/odd indices
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)  # Sin for even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Cos for odd indices

        pe = pe.unsqueeze(0)  # Add batch dimension
        self.register_buffer('pe', pe)  # Store as a buffer (not trainable)

    def forward(self, x):
        """
        Args:
            x (tensor): Shape (batch, seq_len, embed_dim)
        Returns:
            tensor: Positionally encoded input
        """
        return x + 0.1 * self.pe[:, :x.size(1), :]


### Time-Series Transformer Encoder with Positional Encoding
class TimeSeriesModel(nn.Module):
    def __init__(self, num_time_series_features, seq_length, dropout=0.1):
        super(TimeSeriesModel, self).__init__()

        # Projection Layer (Linear transformation for input features)
        self.proj = nn.Linear(num_time_series_features, 64)

        # Positional Encoding
        self.positional_encoding = PositionalEncoding(embed_dim=64, max_len=seq_length)

        # Transformer Encoder Layer
        encoder_layer = nn.TransformerEncoderLayer(d_model=64, nhead=4, dim_feedforward=128, dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)

        # Fully Connected Output Layer
        self.fc = nn.Sequential(
            nn.Linear(seq_length * 64, 128),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change shape to (batch, seq_len, num_features)
        x = self.proj(x)  # Project input features to embedding space
        x = self.positional_encoding(x)  # Add positional encoding
        x = self.transformer(x)  # Pass through Transformer Encoder
        x = x.reshape(x.shape[0], -1)  # Flatten for fully connected layer
        return self.fc(x)  # Fully Connected Output

### Static Feature Model with Dropout
class StaticModel(nn.Module):
    def __init__(self, num_static_features, dropout=0.3):
        super(StaticModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(num_static_features, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.fc(x)

### Text Model (DistilBERT) with Dropout
class TextModel(nn.Module):
    def __init__(self, dropout=0.3):
        super(TextModel, self).__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.fc = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

    def forward(self, text_inputs):
        text_out = self.bert(**text_inputs).last_hidden_state[:, 0, :]
        return self.fc(text_out)

### CNN Model for LiDAR Images (BatchNorm + Dropout)
class LiDARCNN(nn.Module):
    def __init__(self, dropout=0.3):
        super(LiDARCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 56 * 56, 128),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        return self.fc(x)

### Final Multimodal Model with Cross-Attention
class CrossAttentionFusionModel(nn.Module):
    def __init__(self, num_time_series_features, seq_length, num_static_features, dropout=0.3):
        super(CrossAttentionFusionModel, self).__init__()

        self.time_series_model = TimeSeriesModel(num_time_series_features, seq_length, dropout)
        self.static_model = StaticModel(num_static_features, dropout)
        self.text_model = TextModel(dropout)
        self.image_model = LiDARCNN(dropout)

        # Cross-Attention Layers
        self.attn_text_time = CrossAttention(embed_dim=128, num_heads=4, dropout=dropout)  # Text <-> Time-Series
        self.attn_static_image = CrossAttention(embed_dim=128, num_heads=4, dropout=dropout)  # Static <-> Image
        self.attn_time_static = CrossAttention(embed_dim=128, num_heads=4, dropout=dropout)  # Time-Series <-> Static

        # Second-Level Cross-Attention
        self.attn_fusion1 = CrossAttention(embed_dim=128, num_heads=4, dropout=dropout)  # Between attended_text_time and attended_static_image
        self.attn_fusion2 = CrossAttention(embed_dim=128, num_heads=4, dropout=dropout)  # Between attended_time_static and attended_static_image



        # Learnable Attention-Based Fusion Weights
        self.attn_weights = nn.Linear(128 * 3, 3)

        # Final Fusion Layer
        self.fusion_layer = nn.Sequential(
            nn.Linear(128, 1)
        )



    def forward(self, time_series, static_features, text_inputs, image_data):
        time_series_out = self.time_series_model(time_series)
        static_out = self.static_model(static_features)
        text_out = self.text_model(text_inputs)
        image_out = self.image_model(image_data)

        # Apply cross-attention (Text ↔ Time-Series, Static ↔ Image, Time-Series ↔ Static)
        attended_text_time = self.attn_text_time(text_out, time_series_out)
        attended_static_image = self.attn_static_image(static_out, image_out)
        attended_time_static = self.attn_time_static(time_series_out, static_out)

        # Apply second-level cross-attention for deeper interaction
        attended_fusion1 = self.attn_fusion1(attended_text_time, attended_static_image)
        attended_fusion2 = self.attn_fusion2(attended_time_static, attended_static_image)

        # Weighted Fusion
        weights = torch.softmax(self.attn_weights(torch.cat([attended_fusion1, attended_fusion2, attended_static_image], dim=1)), dim=1)
        fused_weighted = (weights[:, 0].unsqueeze(1) * attended_fusion1 +
                          weights[:, 1].unsqueeze(1) * attended_fusion2 +
                          weights[:, 2].unsqueeze(1) * attended_static_image)

        return self.fusion_layer(fused_weighted)



In [ ]:
import gc

del model
del optimizer
del train_loader

gc.collect()
torch.cuda.empty_cache()

torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_max_memory_cached()

In [ ]:
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # For multi-GPU
np.random.seed(SEED)
random.seed(SEED)

# Create PyTorch DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2,
                          worker_init_fn=worker_init_fn, pin_memory=True, persistent_workers=True, prefetch_factor=2)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

# Initialize model
num_time_series_features = 8
seq_length = len(x_train.iloc[0, 0])  # Adjust based on your time-series length
num_static_features = len(x_train.iloc[0, 8:-1])  # Adjust based on your dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CrossAttentionFusionModel(num_time_series_features, seq_length, num_static_features, dropout=0.3).to(device)

# Loss function & optimizer
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)  # Using BCEWithLogitsLoss
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

# Best validation loss tracking
best_val_loss = float("inf")
checkpoint_path = "best_multimodal_cross_attention_model.pth"

# takes 1.5 min per epoch with A100
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

In [ ]:
# Load the best model
model.load_state_dict(torch.load(checkpoint_path, weights_only=True))
print("Loaded best model for evaluation!")

# Evaluate on test data
all_labels, all_preds, all_probs = evaluate_model(model, test_loader, device)

In [ ]:
### Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
cm_rates = cm.astype("float") / cm.sum(axis=1, keepdims=True)

plt.figure(figsize=(6, 5))
sns.heatmap(cm_rates, annot=True, fmt=".2f", cmap="Blues",
                xticklabels=["No Delinquency", "Delinquent"],
                yticklabels=["No Delinquency", "Delinquent"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Normalized Confusion Matrix (Rates)")
plt.savefig(os.path.join(bookPath, "multimodal_confusion_matrix_crossattention.pdf"))

plt.show()

In [ ]:
### ROC Curve & AUC Score
fpr, tpr, _ = roc_curve(all_labels, all_probs)
auc_score = roc_auc_score(all_labels, all_probs)

plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc_score:.3f})", linewidth=2)
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")  # Diagonal reference line
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve - Delinquency Prediction")
plt.legend(loc=4)
plt.grid()
plt.savefig(os.path.join(bookPath, "multimodal_roc_curve_crossattention.pdf"))

plt.show()

### Classification Report
print("Classification Report:\n", classification_report(all_labels, all_preds, digits=4))

We observe an improvement in the ROC AUC score.